In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import balanced_accuracy_score,precision_score,recall_score,f1_score, roc_auc_score

In [2]:
validation_games = [
'ESPORTSTMNT02_3041846',
'ESPORTSTMNT02_3041862',
'ESPORTSTMNT02_3041903',
'ESPORTSTMNT02_3041937',
'ESPORTSTMNT02_3041983',
'ESPORTSTMNT02_3041990',
'ESPORTSTMNT02_3042003',
'ESPORTSTMNT02_3042025',
'ESPORTSTMNT02_3042187',
'ESPORTSTMNT02_3042251',
'ESPORTSTMNT02_3052761',
'ESPORTSTMNT02_3052797',
'ESPORTSTMNT02_3042301',
'ESPORTSTMNT02_3052843',
'ESPORTSTMNT02_3052853',
'ESPORTSTMNT02_3052865',
'ESPORTSTMNT02_3053150',
'ESPORTSTMNT02_3053187',
'ESPORTSTMNT02_3053204',
'ESPORTSTMNT02_3053238',
'ESPORTSTMNT02_3053254',
'ESPORTSTMNT02_3053274',
'ESPORTSTMNT02_3053295',
'ESPORTSTMNT02_3053311',
'ESPORTSTMNT02_3042846',
'ESPORTSTMNT02_3042864',
'ESPORTSTMNT02_3042881',
'ESPORTSTMNT02_3042903',
'ESPORTSTMNT02_3042921',
'ESPORTSTMNT02_3042943',
'ESPORTSTMNT02_3042965',
'ESPORTSTMNT02_3042987',
'ESPORTSTMNT02_3043190',
'ESPORTSTMNT02_3043245',
'ESPORTSTMNT02_3043269',
'ESPORTSTMNT02_3043280',
'ESPORTSTMNT02_3043302',
'ESPORTSTMNT02_3043333',
'ESPORTSTMNT02_3043345',
'ESPORTSTMNT02_3043363',
'ESPORTSTMNT02_3043656',
'ESPORTSTMNT02_3043674',
'ESPORTSTMNT02_3043685',
'ESPORTSTMNT02_3043709',
'ESPORTSTMNT02_3043731',
'ESPORTSTMNT02_3043753',
'ESPORTSTMNT02_3043758',
'ESPORTSTMNT02_3045120',
'ESPORTSTMNT02_3045181',
'ESPORTSTMNT02_3045196',
'ESPORTSTMNT02_3045209',
'ESPORTSTMNT02_3045232',
'ESPORTSTMNT02_3045259',
'ESPORTSTMNT02_3045416',
'ESPORTSTMNT02_3045459',
'ESPORTSTMNT02_3045472',
'ESPORTSTMNT02_3045480',
'ESPORTSTMNT02_3045503',
'ESPORTSTMNT02_3045518',
'ESPORTSTMNT02_3045836',
'ESPORTSTMNT02_3045853',
'ESPORTSTMNT02_3045864',
'ESPORTSTMNT02_3045877',
'ESPORTSTMNT02_3056381',
'ESPORTSTMNT02_3056394',
'ESPORTSTMNT02_3056626',
'ESPORTSTMNT02_3056652',
'ESPORTSTMNT02_3056673',
'ESPORTSTMNT02_3056681',
'ESPORTSTMNT02_3056705',
'ESPORTSTMNT02_3056721',
'ESPORTSTMNT02_3057453',
'ESPORTSTMNT02_3057478',
'ESPORTSTMNT02_3057490',
'ESPORTSTMNT02_3057502',
'ESPORTSTMNT02_3057505',
'ESPORTSTMNT02_3057516',
'ESPORTSTMNT02_3057653',
'ESPORTSTMNT02_3057673',
'ESPORTSTMNT02_3057686',
'ESPORTSTMNT02_3057715',
'ESPORTSTMNT02_3057724',
'ESPORTSTMNT02_3057747',
'ESPORTSTMNT02_3047499',
'ESPORTSTMNT02_3057859',
'ESPORTSTMNT02_3057882',
'ESPORTSTMNT02_3057894',
'ESPORTSTMNT02_3057911',
'ESPORTSTMNT02_3057916',
'ESPORTSTMNT02_3057923',
'ESPORTSTMNT02_3057927',
'ESPORTSTMNT02_3047802',
'ESPORTSTMNT02_3047839',
'ESPORTSTMNT02_3047847',
'ESPORTSTMNT02_3047861',
'ESPORTSTMNT02_3047878',
'ESPORTSTMNT02_3047898',
'ESPORTSTMNT02_3047918',
'ESPORTSTMNT02_3048340',
'ESPORTSTMNT02_3048354',
'ESPORTSTMNT02_3048364',
'ESPORTSTMNT02_3048468',
'ESPORTSTMNT02_3048492',
'ESPORTSTMNT02_3048494',
'ESPORTSTMNT02_3048616',
'ESPORTSTMNT02_3048629',
'ESPORTSTMNT02_3048640',
'ESPORTSTMNT02_3048649',
'ESPORTSTMNT02_3048651',
'ESPORTSTMNT02_3048763',
'ESPORTSTMNT02_3048774',
'ESPORTSTMNT02_3048779',
'ESPORTSTMNT02_3048784',
'ESPORTSTMNT02_3048787',
'ESPORTSTMNT02_3059738',
'ESPORTSTMNT02_3059755',
'ESPORTSTMNT02_3059756',
'ESPORTSTMNT02_3059758',
'ESPORTSTMNT02_3059851',
'ESPORTSTMNT02_3059852',
'ESPORTSTMNT02_3059853',
'ESPORTSTMNT02_3059854'
]

In [3]:
df = pd.read_csv('../data/crawler/unified-events-statistics.csv')
df = df.replace(['BLUE: first_blood',
    'BLUE: dragon',
    'BLUE: herald',
    'BLUE: first_tower_top',
    'BLUE: first_tower_mid',
    'BLUE: first_tower_bot',
    'BLUE: second_tower_top',
    'BLUE: second_tower_mid',
    'BLUE: second_tower_bot',
    'BLUE: third_tower_top',
    'BLUE: third_tower_mid',
    'BLUE: third_tower_bot',
    'BLUE: inhibitor_top',
    'BLUE: inhibitor_mid',
    'BLUE: inhibitor_bot',
    'BLUE: baron',
    'BLUE: elder_dragon',
    'BLUE: nexus_tower',
    'BLUE: nexus',
    'RED: first_blood',
    'RED: dragon',
    'RED: herald',
    'RED: first_tower_top',
    'RED: first_tower_mid',
    'RED: first_tower_bot',
    'RED: second_tower_top',
    'RED: second_tower_mid',
    'RED: second_tower_bot',
    'RED: third_tower_top',
    'RED: third_tower_mid',
    'RED: third_tower_bot',
    'RED: inhibitor_top',
    'RED: inhibitor_mid',
    'RED: inhibitor_bot',
    'RED: baron',
    'RED: elder_dragon',
    'RED: nexus_tower',
    'RED: nexus'], range(38))
# validation_df = df.apply(lambda row: row[df['game'].isin(validation_games)])
# df = df[~df.game.isin(validation_games)]
df

,golId,game,event1,event2,event3,event4,event5,event6,event7,event8,...,redMidGP,redMidWR,redMidKDA,redAdcGP,redAdcWR,redAdcKDA,redSupportGP,redSupportWR,redSupportKDA,result
0,36889,ESPORTSTMNT02_2557053,19,1,2,5,1,20,21,4,...,0,0.00,0.0,1,0.00,0.4,0,0.0,0.0,1
1,36908,ESPORTSTMNT02_2577492,0,20,2,1,24,4,2,5,...,0,0.00,0.0,1,1.00,12.0,2,0.5,3.8,0
2,36932,ESPORTSTMNT05_2530954,19,2,20,5,1,2,3,4,...,0,0.00,0.0,2,0.00,1.7,0,0.0,0.0,1
3,36933,ESPORTSTMNT05_2530886,0,21,20,22,24,21,20,3,...,0,0.00,0.0,2,1.00,31.0,1,1.0,4.7,0
4,36953,ESPORTSTMNT02_2556972,0,21,1,5,2,24,1,4,...,1,0.00,1.4,3,0.33,3.3,0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
437,42963,ESPORTSTMNT04_2450059,0,2,1,4,5,2,1,8,...,4,0.75,6.4,1,1.00,14.0,0,0.0,0.0,1
438,40921,ESPORTSTMNT06_2470519,1,2,0,20,21,5,22,1,...,1,0.00,1.7,3,0.67,11.0,1,1.0,6.0,1
439,40424,ESPORTSTMNT05_2750365,0,2,3,1,4,1,2,5,...,1,0.00,1.0,0,0.00,0.0,0,0.0,0.0,1
440,42494,ESPORTSTMNT06_2530775,0,20,21,20,22,5,21,23,...,2,1.00,14.5,0,0.00,0.0,2,1.0,11.3,0


In [4]:
y = df['result'].copy()
X = df.drop(['golId','result','game','event4','event5','event6','event7','event8','event9','event10','event11','event12','event13','event14','event15','event16','event17','event18','event19','event20','event21','event22','event23','event24','event25','event26','event27','event28','event29','event30','event31','event32','event33','event34','event35','event36','event37','event38','event39','event40','event41','event42','event43','event44','event45'],axis=1)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
X

,event1,event2,event3,blueTopGP,blueTopWR,blueTopKDA,blueJungleGP,blueJungleWR,blueJungleKDA,blueMidGP,blueMidWR,blueMidKDA,blueADCGP,blueADCWR,blueADCKDA,blueSupportGP,blueSupportWR,blueSupportKDA,redTopGP,redTopWR,redTopKDA,redJungleGP,redJungleWR,redJungleKDA,redMidGP,redMidWR,redMidKDA,redAdcGP,redAdcWR,redAdcKDA,redSupportGP,redSupportWR,redSupportKDA
0,19,1,2,1,0.00,0.9,0,0.00,0.0,1,1.00,5.0,3,0.67,6.7,3,0.67,4.8,0,0.00,0.0,0,0.00,0.0,0,0.00,0.0,1,0.00,0.4,0,0.00,0.0
1,0,20,2,3,0.67,5.0,1,0.00,0.7,1,1.00,9.0,0,0.00,0.0,0,0.00,0.0,2,0.00,2.8,0,0.00,0.0,0,0.00,0.0,1,1.00,12.0,2,0.50,3.8
2,19,2,20,0,0.00,0.0,0,0.00,0.0,5,0.60,4.7,0,0.00,0.0,0,0.00,0.0,1,0.00,0.0,0,0.00,0.0,0,0.00,0.0,2,0.00,1.7,0,0.00,0.0
3,0,21,20,0,0.00,0.0,0,0.00,0.0,1,0.00,0.2,0,0.00,0.0,0,0.00,0.0,3,1.00,4.1,2,0.50,11.7,0,0.00,0.0,2,1.00,31.0,1,1.00,4.7
4,0,21,1,4,0.50,2.2,1,0.00,0.5,0,0.00,0.0,0,0.00,0.0,0,0.00,0.0,1,0.00,2.0,1,0.00,1.6,1,0.00,1.4,3,0.33,3.3,0,0.00,0.0
5,0,1,2,1,0.00,0.6,0,0.00,0.0,5,0.80,4.1,6,0.67,17.0,2,0.50,3.4,0,0.00,0.0,3,0.33,1.7,6,0.50,5.3,3,0.33,3.7,0,0.00,0.0
6,0,20,21,3,0.67,4.3,6,0.67,6.1,4,0.75,4.2,5,0.60,5.8,4,1.00,3.3,1,0.00,1.0,0,0.00,0.0,2,0.50,2.8,0,0.00,0.0,2,0.50,3.3
7,19,2,20,1,1.00,3.7,2,1.00,4.3,1,1.00,4.0,1,1.00,18.0,0,0.00,0.0,1,1.00,7.0,1,0.00,4.5,0,0.00,0.0,4,0.50,3.8,1,1.00,8.0
8,20,19,2,4,0.75,4.3,2,1.00,7.0,0,0.00,0.0,0,0.00,0.0,0,0.00,0.0,1,0.00,0.0,0,0.00,0.0,2,1.00,12.5,3,0.00,0.8,3,0.00,1.2
9,0,2,5,0,0.00,0.0,2,1.00,16.0,1,1.00,12.0,0,0.00,0.0,0,0.00,0.0,0,0.00,0.0,5,0.60,5.2,3,0.33,1.7,1,1.00,3.0,0,0.00,0.0


In [5]:
def preprocess_input(X,y):
    X = X.copy()
    X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=42, stratify=y)
    scaler = StandardScaler()   
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    return X_train,X_test,y_train,y_test
X_train,X_test,y_train,y_test = preprocess_input(X,y)

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier

models = {
    'Logistic Regression': LogisticRegression(random_state=42,max_iter=50000),
    'Support Vector Machine (Linear Kernel)': LinearSVC(random_state=42,max_iter=50000),
    'Support Vector Machine (RBF Kernel)': SVC(random_state=42,C=100,gamma=0.001,kernel='rbf',max_iter=50000),
    'Decission Tree': DecisionTreeClassifier(random_state=42),
    'Adaboost': AdaBoostClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42,max_depth=25,n_estimators=600,min_samples_split=2,min_samples_leaf=1),
    'Gradient Boosting Classifier': GradientBoostingClassifier(random_state=42,criterion='friedman_mse',learning_rate=0.025,loss='deviance',max_depth=4,max_features='log2',min_samples_leaf=8,min_samples_split=3, n_estimators=100,subsample=0.5)
}


for name, model in models.items():
  model.fit(X_train,y_train)
  print(name + ' trained')

Logistic Regression trained
Support Vector Machine (Linear Kernel) trained
Support Vector Machine (RBF Kernel) trained
Decission Tree trained
Adaboost trained
Random Forest trained
Gradient Boosting Classifier trained


In [7]:
scores_list = []

for name,model in models.items():    
    scores_list.append({
    'Model': name,
    'Balanced Accuracy': balanced_accuracy_score(y_test,model.predict(X_test)),
    'Precision':  precision_score(y_test,model.predict(X_test)),
    'Recall': recall_score(y_test,model.predict(X_test)),
    'F1-Score': f1_score(y_test,model.predict(X_test)),
    'ROC/AUC Score': roc_auc_score(y_test,model.predict(X_test))
    })
scores = pd.DataFrame(scores_list)

In [8]:
scores

,Model,Balanced Accuracy,Precision,Recall,F1-Score,ROC/AUC Score
0,Logistic Regression,0.547776,0.540541,0.465116,0.500000,0.547776
1,Support Vector Machine (Linear Kernel),0.547776,0.540541,0.465116,0.500000,0.547776
2,Support Vector Machine (RBF Kernel),0.535389,0.529412,0.418605,0.467532,0.535389
3,Decission Tree,0.539181,0.522727,0.534884,0.528736,0.539181
4,Adaboost,0.590495,0.612903,0.441860,0.513514,0.590495
5,Random Forest,0.535389,0.529412,0.418605,0.467532,0.535389
6,Gradient Boosting Classifier,0.592770,0.594595,0.511628,0.550000,0.592770


In [ ]:
import pickle

with open("../models/model-3.pkl", "wb") as f:
    pickle.dump(models['Random Forest'], f)